# Struct

## 类型参数化

在定义 Struct 时，尽可能通过类型参数化限定内部成员的类型；并在限定类型时，尽量采用具体类型而不是抽象类型

## 构造新类

定义一个表示二维平面上点的类，具有 x 坐标和 y 坐标

In [10]:
"""
Represents a 2D point. 

fields: x, y

x, y 类型相同，且都是 Real 的子类型
"""
struct Point{T<:Real}
    x::T
    y::T
end

Point

In [11]:
Point{Int64} <: Point

true

In [12]:
Point{Float64} <: Point

true

### 默认构造函数

新建 struct 时不写自定义内部构造函数，就会使用默认的构造函数

In [6]:
p = Point(3.0, 4.0) # 隐式指定子类型，在实例化过程中进行自动的类型推断

Point{Float64}(3.0, 4.0)

In [2]:
p = Point{Float64}(3, 4) # 显式指定子类型

Point{Float64}(3.0, 4.0)

Point 并非只是一个类型，下面有很多子类型。实例化时，根据传入的参数自动选择最“窄”的类型，以加速运算

所以，上面的定义方式就比
```julia
struct Point
    x::Real
    y::Real
end
```
要好，因为后者的实例永远是 Real 型

### 自定义外部构造函数

In [6]:
"""
这是一个复制构造函数，返回 p 的一个副本
"""
function Point(p)
    Point(p.x, p.y)
end

Point

In [7]:
p2 = Point(p)

Point{Float64}(3.0, 4.0)

In [8]:
methods(Point)

# 2 methods for type constructor:
[1] Point(x::T, y::T) where T<:Real in Main at In[1]:9
[2] Point(p) in Main at In[6]:4

### 自定义内部构造函数

会顶替默认构造函数

In [9]:

struct Point2
    x::Int64
    y::Int64
    function Point2(x=0, y=0) # 自定义内部构造函数
        new(x, y) # `new`关键字
    end
end

Point2()

Point2(0, 0)

### 自定义显示函数

为 `Base.show(io, ...)` 添加一个方法，便于调试

In [10]:
using Printf

function Base.show(io::IO, p::Point)
   @printf(io, "Point: x = %.2f, y = %.2f", p.x, p.y)
end

p3 = Point(1.0, 2.0)

Point: x = 1.00, y = 2.00

### 操作符重载

[Operators Overloading](Basic-Grammar.md#Operators%20Overloading)

In [11]:
import Base.+

function +(p1::Point, p2::Point)
   Point(p1.x + p2.x, p1.y + p2.y) 
end

+ (generic function with 209 methods)

In [12]:
p2 + p3

Point: x = 4.00, y = 6.00

## 常用操作

### 种属判断

In [13]:
p isa Point

true

### 访问字段

In [14]:
p.x

3.0

In [15]:
p.y

4.0

In [16]:
dis = sqrt(p.x^2 + p.y^2)

5.0

### 查看字段

`fieldnames(stru::Struct)` 查看 struct 的字段，返回 tuple{Symbol}

In [17]:
fieldnames(Point)

(:x, :y)

`isdefined()` 对象是否有某个字段

In [18]:
isdefined(p, :x)

true

In [19]:
isdefined(p, :z)

false

## mutable Struct

Struct 是不可变对象，一旦构造，字段值不能被重新赋值

除非专门用 `mutable struct` 声明一个可变的 struct

In [20]:
p.x = 2 # 实例化后不可修改

LoadError: setfield!: immutable struct of type Point cannot be changed

In [21]:
mutable struct MPoint
  x::Float64
  y::Float64
end

In [22]:
p3 = MPoint(1, 2)

MPoint(1.0, 2.0)

In [23]:
p3.x

1.0

In [24]:
p3.x = 2

2

In [25]:
p3

MPoint(2.0, 2.0)

In [26]:
"""
Represents a rectangle.

fields: width, height, corner.
"""
struct Rectangle
    width
    height
    corner # corner 是一个 Point 类的对象
end

Rectangle

In [27]:
origin = MPoint(0.0, 0.0)

MPoint(0.0, 0.0)

In [28]:
box = Rectangle(100.0, 200.0, origin)

Rectangle(100.0, 200.0, MPoint(0.0, 0.0))

### deepcopy()

实现深拷贝

## 多重派发与 Struct

**多重派发**允许同一个函数针对不同的类型执行不同的操作

In [29]:
function dist1(p1::Point, p2::Point)
  sqrt((p1.x-p2.x)^2+(p1.y-p2.y)^2) # 两个点的欧氏距离
end

dist1 (generic function with 1 method)

In [30]:
p1 = Point(1, 2)

Point: x = 1.00, y = 2.00

In [31]:
p2 = Point(2, 1)

Point: x = 2.00, y = 1.00

In [32]:
dist1(p1, p2)

1.4142135623730951

In [33]:
dist1 # 函数有一个方法

dist1 (generic function with 1 method)

增加一个方法：

In [34]:
function dist1(p1::Point{Int64}, p2::Point{Int64})
  abs(p1.x-p2.x) + abs(p1.y-p2.y) # 两个点的曼哈顿距离
end

dist1 (generic function with 2 methods)

In [35]:
dist1 # 函数有了两个方法

dist1 (generic function with 2 methods)

In [36]:
dist1(Point(1, 1), Point(0, 0)) # 只有两个点均为整数点，才返回曼哈顿距离

2

In [37]:
dist1(Point(1.0, 1.0), Point(0.0, 0.0))

1.4142135623730951

In [38]:
dist1(Point(1.0, 1.0), Point(0, 0))

1.4142135623730951

In [39]:
+ # `+`() 有208个方法！

+ (generic function with 209 methods)

# 实例

从 Fox 和 Chicken 这两种 Animal 开始，定义 struct 和相关函数，并在后来需要时扩展

In [40]:
abstract type Animal end # 定义抽象类型

struct Fox <: Animal # 定义两个基于 Animal 的具体类
  weight::Float64
end

struct Chicken <: Animal
  weight::Float64
end

In [41]:
# 实例化两个动物
fiona = Fox(4.2); 
big_bird = Chicken(2.9);

In [42]:
combined_weight(A1::Animal, A2::Animal) = A1.weight + A2.weight # 加总函数

combined_weight (generic function with 1 method)

In [43]:
"""
trouble(A::Animal, B::Animal) -> Bool

两个动物相处，是否会有 trouble
"""
function trouble(A::Animal, B::Animal)
  if A isa Fox && B isa Chicken 
    return true # 狐狸会吃鸡，自然会有 trouble
  elseif A isa Chicken && B isa Fox
    return true
  elseif A isa Chicken && B isa Chicken
    return false # 两只鸡在一起不会有 trouble
  end
end

trouble

In [44]:
trouble(fiona, big_bird)

true

## 扩展对函数 `trouble()` 的定义

In [45]:
trouble(F::Fox, C::Chicken) = true
trouble(C::Chicken, F::Fox) = true
trouble(C1::Chicken, C2::Chicken) = false

trouble (generic function with 4 methods)

In [46]:
methods(trouble)

# 4 methods for generic function "trouble":
[1] trouble(F::Fox, C::Chicken) in Main at In[45]:1
[2] trouble(C::Chicken, F::Fox) in Main at In[45]:2
[3] trouble(C1::Chicken, C2::Chicken) in Main at In[45]:3
[4] trouble(A::Animal, B::Animal) in Main at In[43]:6

In [47]:
dora = Chicken(2.2)
trouble(dora, big_bird)

false

## 扩展 Animal 新的子类

可以在其他包中定义新的 Animal 子类 Zebra，然后扩展 `trouble()` 使之能接收新的 Zebra 类型

In [48]:
struct Zebra <: Animal
  weight::Float64
end

# 斑马与狐狸、鸡在一起都没问题
trouble(F::Fox, Z::Zebra) = false 
trouble(Z::Zebra, F::Fox) = false
trouble(C::Chicken, Z::Zebra) = false
trouble(Z::Zebra, F::Fox) = false

trouble (generic function with 7 methods)

In [49]:
marty = Zebra(412)
trouble(big_bird, marty)

false

而且，不需要额外定义任何函数即可计算 Zebra 和其他动物的重量之和

In [50]:
combined_weight(big_bird, marty)

414.9